Hellan Herrmann Johnson Method
====

In [ ]:
from ngsolve import *
from netgen.geom2d import unit_square
import netgen.gui
%gui tk

Define product finite element space for $H(\text{div div}) \times H^1$

In [ ]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

order = 2
Sigma = HDivDiv(mesh, order=order)
W = H1(mesh, order=order+1, dirichlet="top|bottom|left|right")
X = FESpace( [Sigma,W] )

print ("ndof-Sigma:", Sigma.ndof, ", ndof-W:", W.ndof, ", ndof-X:", X.ndof)

$$A(\sigma,u; \tau, v) = -a(\sigma,\tau) + b(\sigma,v) + b(\tau, u)$$
with
\begin{align*}
a(\sigma,\tau) & = \int_\Omega \sigma: \tau\,dx \\
b(\sigma, v) & = \sum_T \int_T \text{div} \sigma \cdot \nabla v\,dx - \int_{\partial T} \sigma_n\cdot \nabla_t v\,ds
\end{align*}

In [ ]:
sigma, u = X.TrialFunction()
tau, v = X.TestFunction()

n = specialcf.normal(2)
def tang(u): return u-(u*n)*n

a = BilinearForm(X, symmetric=True)
a += SymbolicBFI (-InnerProduct (sigma, tau) + div(sigma)*grad(v) 
                  + div(tau)*grad(u))
a += SymbolicBFI (-(sigma*n) * tang(grad(v)) - (tau*n)*tang(grad(u)), 
                  element_boundary=True)
a.Assemble()

f = LinearForm(X)
f += SymbolicLFI (500 * v)
f.Assemble()

u = GridFunction(X)
u.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec

Draw (u.components[0], mesh, "sigma")
Draw (u.components[1], mesh, "disp")